In [1]:
import pandas as pd
import numpy as np

import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Modul_6_Statistics/7_cars.csv')
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [8]:
# Посмотрим на данные
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [18]:
# Создадим новый признак – марку автомобиля 
df['mark'] = df.CarName.apply(lambda x: x.split(' ')[0])
df.drop(columns=['CarName', 'car_ID'])
df['mark'].nunique()

28

In [19]:
# Посмотрим на уникальные значения mark
df['mark'].value_counts()

toyota         31
nissan         17
mazda          15
honda          13
mitsubishi     13
subaru         12
peugeot        11
volvo          11
volkswagen      9
dodge           9
bmw             8
buick           8
audi            7
plymouth        7
saab            6
isuzu           4
porsche         4
alfa-romero     3
jaguar          3
chevrolet       3
vw              2
maxda           2
renault         2
porcshce        1
vokswagen       1
toyouta         1
Nissan          1
mercury         1
Name: mark, dtype: int64

In [33]:
# Исправим ошибки и еще раз посмотрим сколько уникальных марок  
df = df.replace({'mark' : { 'maxda' : 'mazda',
    'Nissan' : 'nissan',
    'porcshce' : 'porsche',
    'toyouta' : 'toyota',
    'vw' : 'volkswagen',
    'vokswagen' : 'volkswagen' }})

In [35]:
df['mark'].nunique()

22

In [37]:
# Оставим часть предикторов
regression_1 = df[['price', 'mark', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower']]

In [38]:
regression_1

,price,mark,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114
201,19045.0,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160
202,21485.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134
203,22470.0,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106


In [47]:
# Посмотрим корреляцию между price и другими переменными. 
regression_1.corr().round(2)

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.00,0.58,0.68,0.76,0.84,0.87,0.55,0.81
wheelbase,0.58,1.00,0.87,0.80,0.78,0.57,0.49,0.35
carlength,0.68,0.87,1.00,0.84,0.88,0.68,0.61,0.55
carwidth,0.76,0.80,0.84,1.00,0.87,0.74,0.56,0.64
curbweight,0.84,0.78,0.88,0.87,1.00,0.85,0.65,0.75
enginesize,0.87,0.57,0.68,0.74,0.85,1.00,0.58,0.81
boreratio,0.55,0.49,0.61,0.56,0.65,0.58,1.00,0.57
horsepower,0.81,0.35,0.55,0.64,0.75,0.81,0.57,1.00


In [48]:
regression_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           205 non-null    float64
 1   mark            205 non-null    object 
 2   fueltype        205 non-null    object 
 3   aspiration      205 non-null    object 
 4   carbody         205 non-null    object 
 5   drivewheel      205 non-null    object 
 6   wheelbase       205 non-null    float64
 7   carlength       205 non-null    float64
 8   carwidth        205 non-null    float64
 9   curbweight      205 non-null    int64  
 10  enginetype      205 non-null    object 
 11  cylindernumber  205 non-null    object 
 12  enginesize      205 non-null    int64  
 13  boreratio       205 non-null    float64
 14  horsepower      205 non-null    int64  
dtypes: float64(5), int64(3), object(7)
memory usage: 24.1+ KB


In [50]:
# Для линейной регрессии преобразуем категориальные переменные в фиктивные/индикаторные переменные
df_dummy = pd.get_dummies(data=regression_1[['mark', 'fueltype', 'aspiration','carbody', 'drivewheel', 'enginetype', 'cylindernumber']], drop_first = True)

In [61]:
# Объединим с другими переменными, которые не были преобразованы
reg_2 = pd.concat([regression_1[['price', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower']], df_dummy ], axis=1)
reg_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price                  205 non-null    float64
 1   wheelbase              205 non-null    float64
 2   carlength              205 non-null    float64
 3   carwidth               205 non-null    float64
 4   curbweight             205 non-null    int64  
 5   enginesize             205 non-null    int64  
 6   boreratio              205 non-null    float64
 7   horsepower             205 non-null    int64  
 8   mark_audi              205 non-null    uint8  
 9   mark_bmw               205 non-null    uint8  
 10  mark_buick             205 non-null    uint8  
 11  mark_chevrolet         205 non-null    uint8  
 12  mark_dodge             205 non-null    uint8  
 13  mark_honda             205 non-null    uint8  
 14  mark_isuzu             205 non-null    uint8  
 15  mark_j

Построим модель с одним предиктором: price – horsepower.

In [58]:
results = smf.ols('price ~ horsepower', reg_2).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 24 May 2022   Prob (F-statistic):           1.48e-48
Time:                        15:03:45   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

Модель со всеми предикторами, кроме марок машин

In [65]:
Y = reg_2.price
X = sm.add_constant(reg_2.drop(['price', 'mark_audi', 'mark_bmw',
       'mark_buick', 'mark_chevrolet', 'mark_dodge', 'mark_honda',
       'mark_isuzu', 'mark_jaguar', 'mark_mazda', 'mark_mercury',
       'mark_mitsubishi', 'mark_nissan', 'mark_peugeot', 'mark_plymouth',
       'mark_porsche', 'mark_renault', 'mark_saab', 'mark_subaru',
       'mark_toyota', 'mark_volkswagen', 'mark_volvo'], axis = 1) 
model = sm.OLS(Y, X)  
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Tue, 24 May 2022   Prob (F-statistic):           9.86e-81
Time:                        15:17:06   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

Модель со всеми предикторами

In [66]:
Y = reg_2.price
X = sm.add_constant(reg_2.drop('price', axis = 1)) 

model = sm.OLS(Y, X)  
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Tue, 24 May 2022   Prob (F-statistic):           4.86e-89
Time:                        15:17:58   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

#### Оставляем модель, где меньше предикторов, так как R2 изменился не очень сильно, а часть марок вообще не значима. Выбранная модель объясняет примерно 90 % дисперсии. Среди предикторов 10 из 27 оказались не значимыми (p > 0.05).